In [ ]:
# Change directory to cadmesh repo until conda/pip deployment is ready
import os
os.chdir("/home/sebastian/Projects/ABC/cadmesh/")

In [ ]:
# Process step files
from cadmesh import process_step_folder
results = process_step_folder(data_path="data/abc_0000", output_path="data/results/abc_0000", log_path="data/log_abc", file_pattern="*/*.step", file_range=[0, 10])

In [ ]:
# Analyse log files to check how many files succeeded/failed in step processing
from cadmesh import analyse_log_files
analyse_log_files("data/log_abc/*.log")

In [ ]:
# Process geometry files to find curve and surface types      
from cadmesh import analyse_curve_and_surface_types
curves, surfs, rev_ext = analyse_curve_and_surface_types("data/results/abc_00*/*_geo.yaml")